In [1]:
import os
import json
import requests

import numpy as np
import tqdm

from qcportal import PortalClient
from qcportal.record_models import BaseRecord, RecordStatusEnum

from openff.units import unit

from openff.toolkit.topology import Molecule
from openff.toolkit.utils import OpenEyeToolkitWrapper, ToolkitRegistry

from openff.qcsubmit.common_structures import MoleculeAttributes
from openff.qcsubmit.datasets import BasicDataset
from openff.qcsubmit.factories import BasicDatasetFactory
from openff.qcsubmit.results import BasicResultCollection
from openff.qcsubmit.results.filters import (
    ConnectivityFilter,
    RecordStatusFilter,
    UnperceivableStereoFilter,
    SinglepointRecordFilter,
)

from openff.qcsubmit._pydantic import Field

ADDRESS = "https://api.qcarchive.molssi.org:443/"
client = PortalClient(ADDRESS, cache_dir=".",)

/Users/jenniferclark/mamba/envs/qca-clean-openff/lib/python3.11/site-packages/openff/amber_ff_ports/amber_ff_ports.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## Get v1.0 Datasets

In [5]:
dataset_info = [
    ("MLPepper RECAP Optimized Fragments v1.0", 'wb97x-d/def2-tzvpp'),
    ("MLPepper RECAP Optimized Fragments v1.0 Add Iodines", 'wb97x-d/def2-tzvpp'),
    ("MLPepper RECAP Optimized Fragments v1.0", 'wb97x-d/def2-tzvpp/ddx-water'),
    ("MLPepper RECAP Optimized Fragments v1.0 Add Iodines", 'wb97x-d/def2-tzvpp/ddx-water'),
]
datasets = []
for ds_name, ds_spec in dataset_info:
    datasets.append(BasicResultCollection.from_server(
        client=client,
        datasets=ds_name,
        spec_name=ds_spec,
    ))
    print(f"{datasets[-1].n_results} records in {ds_name} with {ds_spec}")

68966 records in MLPepper RECAP Optimized Fragments v1.0 with wb97x-d/def2-tzvpp
6126 records in MLPepper RECAP Optimized Fragments v1.0 Add Iodines with wb97x-d/def2-tzvpp
68966 records in MLPepper RECAP Optimized Fragments v1.0 with wb97x-d/def2-tzvpp/ddx-water
6131 records in MLPepper RECAP Optimized Fragments v1.0 Add Iodines with wb97x-d/def2-tzvpp/ddx-water


In [6]:
rec_and_mol = []
for ds in datasets:
    rec_and_mol.extend(ds.to_records())
print('Finished converting to records and extracting entries',flush = True)

KeyboardInterrupt: 

# Make New Dataset

In [ ]:
with open("ds_info.json") as f:
    dataset_information = json.load(f)

dataset_factory1 = BasicDatasetFactory()
provenance1 = dataset_factory1.provenance(ToolkitRegistry([OpenEyeToolkitWrapper]))

dataset1 = BasicDataset(
    dataset_name=dataset_information["dataset_name"],
    dataset_tagline=dataset_information["dataset_tagline"],
    description=dataset_information["description"],
    provenance=provenance1
)
dataset1.metadata.submitter = "jaclark5"
dataset1.metadata.long_description_url = dataset_information["metadata.long_description_url"]


In [ ]:
# Have to add records this way to avoid a round trip through the toolkit.
records_by_cmiles= {}
for record, molecule in rec_and_mol:
    cmiles = molecule.to_smiles(isomeric=True, explicit_hydrogens=True, mapped=True)
    if cmiles in records_by_cmiles.keys():
        records_by_cmiles[cmiles].append((record, molecule))
    else:
        records_by_cmiles[cmiles]=[(record, molecule)]

count = 0 # NoteHere
hashes = set()
prev_len, prev_smiles = 0, ""
for records in tqdm.tqdm(records_by_cmiles.values()):
    base_record, base_molecule = records[0]
    base_molecule._conformers = [m.conformers[0] for _, m in records]

    dataset1.add_molecule(
        index=base_molecule.to_smiles(
            isomeric=True, explicit_hydrogens=False, mapped=False
        ),
        molecule=None,
        initial_molecules=[rec.initial_molecule for rec, _ in records],
        attributes=MoleculeAttributes.from_openff_molecule(base_molecule),
        extras=base_record.extras,
        keywords=base_record.specification.keywords,
    )
    
#    current_count = len({
#        qcemol.identifiers.molecule_hash
#        for moldata in dataset1.dataset.values()
#        for qcemol in moldata.initial_molecules
#    })
#    if current_count != count + len(records):
#        print(len(dataset1.dataset), prev_len)
#        print(prev_smiles)
#        print(base_molecule.to_smiles(isomeric=True, explicit_hydrogens=False, mapped=False))
#        print("Same?", prev_smiles == base_molecule.to_smiles(isomeric=True, explicit_hydrogens=False, mapped=False))
#        raise ValueError(f"There should be {count + len(records)}, not {current_count} molecules")
#    else:
#        prev_len = len(dataset1.dataset)
#        prev_smiles = base_molecule.to_smiles(isomeric=True, explicit_hydrogens=False, mapped=False)
#    count += len(records)
#    for rec, _ in records:
#        hashes.add(rec.initial_molecule.identifiers.molecule_hash)

NameError: name 'rec_and_mol' is not defined

In [ ]:
#print(current_count, count, len(records))
# How do we have one fewer molecule than before we tried to add 8...

30463 30464 8


In [ ]:
#new_hashes = [rec.initial_molecule.identifiers.molecule_hash for rec, _ in records]
#test_hashes = hashes.copy().union(new_hashes)

In [ ]:
#print(len(hashes) + len(new_hashes), len(test_hashes))

30472 30472


In [ ]:
#from pprint import pprint
#from qcportal.serialization import encode_to_json
#pprint(encode_to_json(records[0]))

[{'created_on': '2021-04-20T15:11:40.129611+00:00',
  'creator_user': None,
  'energies': [-1354.2532051055496,
               -1354.294529401838,
               -1354.308747123733,
               -1354.3095908944638,
               -1354.3127072999548,
               -1354.3083354516334,
               -1354.3125137884006,
               -1354.3139773390255,
               -1354.314675228994,
               -1354.3150541254315,
               -1354.3155633351791,
               -1354.3159278067233,
               -1354.3164003125705,
               -1354.3166517466643,
               -1354.3167650198143,
               -1354.3168228768332,
               -1354.3168265000822,
               -1354.3168375963983,
               -1354.3168456086757,
               -1354.3168525157914,
               -1354.3168510324685,
               -1354.316847148259,
               -1354.3168519402877,
               -1354.3168540189972,
               -1354.3168499712367,
               -1354.3168480

In [ ]:
# Check that the molecules are identical
opt_hashes = {rec.initial_molecule.get_hash() for rec, _mol in rec_and_mol}

new_hashes = {
    qcemol.identifiers.molecule_hash
    for moldata in dataset1.dataset.values()
    for qcemol in moldata.initial_molecules
}

print('Molecules are the same? ',opt_hashes==new_hashes)

Molecules are the same?  False


## Write Statistics

In [ ]:
n_confs = np.array([mol.n_conformers for mol in dataset1.molecules])
n_heavy_atoms = np.array([mol.to_rdkit().GetNumHeavyAtoms() for mol in dataset1.molecules])
masses = np.array([sum([atom.mass.m for atom in mol.atoms]) for mol in dataset1.molecules])
elements = sorted(set(atom.symbol for mol in dataset1.molecules for atom in mol.atoms))
unique_charges = set()
for mol in dataset1.molecules:
    unique_charges.add(mol.total_charge.m_as(unit.elementary_charge))
unique_charges = sorted(unique_charges)


In [ ]:
print(f"* Number of unique molecules: {dataset1.n_molecules}")
print("* Number of conformers:", dataset1.n_records)
print(
    "* Number of conformers (min, mean, max): {}, {}, {}".format(
        int(min(n_confs)), int(np.mean(n_confs)), int(max(n_confs))
))
print(
    "* Molecular weight (min, mean, max): {:.2f}, {:.2f}, {:.2f}".format(
        min(masses), np.mean(masses), max(masses)
    )
)
print("* Charges: {}".format(", ".join([str(x) for x in unique_charges])))
print("* Elements: {}".format(", ".join([x for x in elements])))

* Number of unique molecules: 9784
* Number of conformers: 74362
* Number of conformers (min, mean, max): 1, 7, 10
* Molecular weight (min, mean, max): 16.04, 348.76, 1105.16
* Charges: -2.0, -1.0, 0.0, 1.0, 2.0
* Elements: Br, C, Cl, F, H, N, O, P, S


In [ ]:

print("* Name: {}".format(dataset1.dataset_name))
print("* Purpose: {}".format(dataset1.dataset_tagline))
print("* Description: {}".format(dataset1.description))
print("* Submitter: {}\n".format(dataset1.metadata.submitter))

print("\n## Metadata")
print(f"* Elements: {{{', '.join(sorted(dataset1.metadata.elements))}}}")

for spec, obj in dataset1.qc_specifications.items():
    od = obj.dict()
    print("* Program:", od["program"])
    print("* QC Specifications:", spec)
    for field, value in od.items():
        if field in ["scf_properties", 'program', "spec_name", "spec_description"]:
            continue
        print(f"  * {field}: {od[field]}")
    print("  * SCF Properties:")
    for field in od["scf_properties"]:
        print(f"    * {field}")


* Name: OpenFF Industry Benchmark Season 1 v1.2
* Purpose: The combination of all publicly chosen compound sets by industry partners from the OpenFF season 1 industry benchmark with unrealistic conformers removed.
* Description: This dataset is the public counterpart of the OpenFF Industry Benchmark Season 1. Each industry partner has selected a range of diverse molecules which represent their current chemical interests. The dataset will be used in conjunction with private counterparts also designed by each partner to give an unbiased assessment of the progress and current performance of the OpenFF line of force fields in comparison with other contemporary force fields.
The v1.1 dataset features corrected Merck (MRK) molecules with explicit hydrogens. The original v1.0 dataset did not have explicit hydrogens on these molecules, resulting in poor starting conformers that have largely failed to geometry optimize under QM.
The v1.1 dataset was prepared from the v1.0 dataset, excising the 

In [ ]:
dataset1.export_dataset("dataset.json.bz2")
dataset1.molecules_to_file('dataset.smi', 'smi')
dataset1.visualize("dataset.pdf", columns=8)